In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import random

unknown_ID = 0
padding_ID = 1
MAX_SENTENCE_LENGTH = 500
NUM_CLASSES = 20

Instructions for updating:
non-resource variables are not supported in the long term


#Tạo class RNN biểu diễn cấu trúc mạng RNN

In [14]:
class RNN:
    def __init__(self,
                vocab_size,
                embedding_size,
                lstm_size,
                batch_size):
        self._vocab_size = vocab_size
        self._lstm_size = lstm_size
        self._embedding_size = embedding_size
        self._batch_size = batch_size

        self._data = tf.placeholder(tf.int32, shape= [batch_size, MAX_SENTENCE_LENGTH])
        self._labels = tf.placeholder(tf.int32, shape= [batch_size, ])
        self._sentence_lengths = tf.placeholder(tf.int32, shape= [batch_size, ])
        self._final_tokens = tf.placeholder(tf.int32, shape= [batch_size, ])

    def embedding_layer(self, indices):
      pretrained_vectors = []
      pretrained_vectors.append(np.zeros(self._embedding_size))
      np.random.seed(2021)
      
      for _ in range (self._vocab_size + 1):
        pretrained_vectors.append(np.random.normal(loc=0., scale=1., size=self._embedding_size))

      pretrained_vectors = np.array(pretrained_vectors)

      self._embedding_matrix = tf.get_variable(
          name='embedding',
          shape= (self._vocab_size + 2, self._embedding_size),
          initializer=tf.constant_initializer(pretrained_vectors)
      )
      return tf.nn.embedding_lookup(self._embedding_matrix, indices)

    def LSTM_layer(self, embeddings):
      # lstm_cell = tf.rnn_cell.BasicLSTMCell(self._lstm_size)
      lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(self._lstm_size)
      zero_state = tf.zeros(shape= (self._batch_size, self._lstm_size))
      initial_state = tf.nn.rnn_cell.LSTMStateTuple(zero_state, zero_state)

      lstm_input = tf.unstack(tf.transpose(embeddings, perm=[1, 0, 2]))

      lstm_output, last_state = tf.nn.static_rnn(
          cell= lstm_cell,
          inputs= lstm_input,
          initial_state= initial_state,
          sequence_length= self._sentence_lengths
      )

      lstm_output = tf.unstack(tf.transpose(lstm_output, perm=[1, 0, 2]))
      lstm_output = tf.concat(lstm_output, axis= 0)

      #self._mask = [num docs * MAX_SENT_LENGTH, ]

      mask = tf.sequence_mask(
          lengths= self._sentence_lengths,
          maxlen= MAX_SENTENCE_LENGTH,
          dtype= tf.float32
      )
      mask = tf.concat(tf.unstack(mask, axis= 0), axis= 0)
      mask = tf.expand_dims(mask, -1)

      lstm_output = mask * lstm_output
      lstm_output_split = tf.split(lstm_output, num_or_size_splits= self._batch_size)
      lstm_output_sum = tf.reduce_sum(lstm_output_split, axis= 1)
      lstm_output_average = lstm_output_sum / tf.expand_dims(
                  tf.cast(self._sentence_lengths, dtype= tf.float32), -1
      )
      return lstm_output_average
    def trainer(self, loss, learning_rate):
      train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
      return train_opt
    
    def build_graph(self):
        embeddings = self.embedding_layer(self._data)
        lstm_output = self.LSTM_layer(embeddings)

        weights = tf.get_variable(
            name= 'final_layer_weights',
            shape= (self._lstm_size, NUM_CLASSES),
            initializer= tf.random_normal_initializer(seed= 2021)
        )
        biases = tf.get_variable(
        name= 'final_layer_biases',
        shape= (NUM_CLASSES),
        initializer= tf.random_normal_initializer(seed= 2021)
        )
        logits = tf.matmul(lstm_output, weights) + biases

        labels_one_hot = tf.one_hot(
            indices= self._labels,
            depth= NUM_CLASSES,
            dtype= tf.float32
        )

        loss = tf.nn.softmax_cross_entropy_with_logits(
            labels= labels_one_hot,
            logits= logits
        )

        loss = tf.reduce_mean(loss)

        probs = tf.nn.softmax(logits)
        predicted_labels = tf.argmax(probs, axis= 1)
        predicted_labels = tf.squeeze(predicted_labels)

        return predicted_labels, loss
    
        

        

#Tạo class DataReader để load dữ liệu vào mạng

In [5]:
class DataReader:
  def __init__(self, data_path, batch_size):
    self._batch_size = batch_size
    
    with open(data_path) as f:
      d_lines = f.read().splitlines()
    self._data = []
    self._labels = []
    self._final_tokens = []
    self._sentence_lengths = []
    for data_id, line in enumerate(d_lines):
        
        features = line.split('<fff>')
        label, doc_id, sentence_lengths = int(features[0]), int(features[1]), int(features[2])
      
        tokens = features[3].split()

        self._data.append(tokens)
        self._labels.append(label)
        self._sentence_lengths.append(sentence_lengths)
        self._final_tokens.append(tokens[-1])
    self._data = np.array(self._data)
    self._labels = np.array(self._labels)
    self._final_tokens = np.array(self._final_tokens)
    self._sentence_lengths = np.array(self._sentence_lengths)

    self._num_epoch = 0
    self._batch_id = 0

  def next_batch(self):
    start = self._batch_id * self._batch_size
    end = start + self._batch_size
    self._batch_id += 1

    if end + self._batch_size > len(self._data):
      end = len(self._data)
      start = end - self._batch_size
      self._num_epoch += 1
      self._batch_id = 0
      indices = list(range(len(self._data)))
      random.seed(2021)
      random.shuffle(indices)
      self._data, self._labels, self._sentence_lengths, self._final_tokens  = self._data[indices], self._labels[indices], self._sentence_lengths[indices], self._final_tokens[indices]
    return self._data[start:end], self._labels[start:end], self._sentence_lengths[start: end], self._final_tokens[start: end]


#Tạo hàm để chạy quá trình train và test 

In [17]:
def train_and_evaluate_RNN():
  with open('/content/drive/MyDrive/DSLab/Session 4/data/w2v_1/vocab-raw.txt') as f:
    vocab_size = len(f.read().splitlines())
  tf.reset_default_graph()
  # tf.random.set_seed(2021)
  tf.set_random_seed(2021)

  rnn = RNN(vocab_size= vocab_size,
            embedding_size= 300,
            lstm_size= 50,
            batch_size= 50)
  predicted_labels, loss = rnn.build_graph()
  # tf.reset_default_graph()
  train_opt = rnn.trainer(loss= loss, learning_rate= 0.01)
  with tf.Session() as sess:
    train_data_reader = DataReader(
        data_path= '/content/drive/MyDrive/DSLab/Session 4/data/w2v_1/20new-train-encoded.txt',
        batch_size= 50,
    )
    test_data_reader = DataReader(
        data_path= '/content/drive/MyDrive/DSLab/Session 4/data/w2v_1/20news-test-encoded.txt',
        batch_size= 50,
    )

    step = 0
    MAX_STEP = 1000

    sess.run(tf.global_variables_initializer())
    while step < MAX_STEP:
      train_next_batch = train_data_reader.next_batch()
      train_data, train_labels, train_sentence_lengths, train_final_tokens = train_next_batch
      plabels_eval, loss_eval, _ = sess.run(
          [predicted_labels, loss, train_opt],
          feed_dict= {
              rnn._data: train_data,
              rnn._labels: train_labels,
              rnn._sentence_lengths: train_sentence_lengths,
              rnn._final_tokens: train_final_tokens
          }
      )
      step += 1
      if step % 20 == 0:
        print('Step: {}, loss: {}'.format(step, loss_eval))
      
      if train_data_reader._batch_id == 0:
        num_true_preds = 0
        while True:
          next_test_batch = test_data_reader.next_batch()
          test_data, test_labels, test_sentence_lengths, test_final_tokens = next_test_batch

          test_plabels_eval = sess.run(
              predicted_labels,
              feed_dict={
                rnn._data: test_data,
                rnn._labels: test_labels,
                rnn._sentence_lengths: test_sentence_lengths,
                rnn._final_tokens: test_final_tokens
              }
          )

          matches = np.equal(test_plabels_eval, test_labels)
          num_true_preds += np.sum(matches.astype(float))

          if test_data_reader._batch_id == 0:
            break
        print('Epoch: ', train_data_reader._num_epoch)
        print('Accuracy on test data: ', num_true_preds * 100. / len(test_data_reader._data))  
    



In [ ]:

train_and_evaluate_RNN()